In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
import plotly.express as px
import ipywidgets as widgets
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

In [ ]:
seed=18453

># 1. DIABETES-DATA

># 1.1. Acceso y almacenamiento de datos



El conjunto de datos que emplearemos es `diabetes`. Proviene del Instituto Nacional de Diabetes y enfermedades digestivas y renales. La base de datos está formada por varias variables predictoras y una variable objetivo,`Outcome`. El objetivo de esta base de datos es predecir si un paciente tiene diabetes o no, basándonos en las variables predictoras.

Por tanto, nuestro conjunto de datos está formado por 8 variables predictoras y una variable objetivo. La variable objetivo `Outcome` consta de 768 instancias, tomando valores de 0 y 1. Nuestas variables predictoras son las siguientes:

*     Número de veces preñada
*     Glucosa
*     Presión Sanguinea
*     Espesor de la piel
*     Insulina
*     Índice de masa corporal
*     Diabetes heredada
*     Años



In [ ]:
filepath= "../input/pima-indians-diabetes-database/diabetes.csv"
target = "Outcome"

data = utils.load_data(filepath,None,target)
data.index=range(data.shape[0])
data.sample(8,random_state=seed)

Como vemos, hemos cargado el conjunto de datos y hemos mostrado 8 instancias, obtenidas de forma aleatoria para evitar una muestra sesgada. A continuación, obtendremos nuestras variables predictoras por un lado y la variable objetivo por otro, con el fin de obtener una muestra de entrenamiento y una muestra de prueba.

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")
X.sample(5,random_state=seed)

In [ ]:
y.sample(5,random_state=seed)

In [ ]:
train_size= 0.7
(X_train,X_test,y_train,y_test)= train_test_split(X,y,stratify=y,
                                                  random_state=seed,train_size=train_size)


Una vez dividido nuestro conjunto de datos en entrenamiento y prueba, nos aseguraremos que funciona correctamente. Para ello, utilizaremos siempre la muestra de entrenamiento, evitando así una fuga de datos.


In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

Por último, uniremos las variables predictos con la variable clase, lo cual nos facilitará realizar operaciones más adelante.

In [ ]:
data_train= utils.join_dataset(X_train,y_train)
data_test= utils.join_dataset(X_test,y_test)

Y mostramos el data_train para comprobar si se ha realizado correctamente la unión.

In [ ]:
data_train.sample(10,random_state=seed)

># 1.2. Análisis exploratorio de datos

### Descripción del conjunto de datos

En primer lugar, profundizaremos en la descripción del conjunto de datos.

In [ ]:
data_train.shape

Observamos que nuestro conjunto de datos, efectivamente está formado por 9 variables, 8 predictoras y 1 objetivo. Además, obtendemos la información de que tenemos 537 casos.

In [ ]:
data_train.info(memory_usage=False)

In [ ]:
data_train.isnull().sum()

In [ ]:
y_train.cat.categories

Vemos el tipo de nuestras variables. Observamos que las variables predictoras, de la muestra entrenamiento, son variables continuas. Además, podemos observar que todas tienen 537 instancias, que corresponde al 70% de la muestra de entrenamiento, por tanto, ninguna variable tiene valores perdidos. Por último, vemos que nuestra variable objetivo es una variable continua, con valores 0 y 1.

### Visualización de las variables

**Estudio de histogramas**

En segundo lugar, proseguimos con un análisis de nuestras variables. Esto nos servirá para identificar posibles outliers y falicitarnos la tarea de preprocesar datos. Comenzaremos mostrando las distribuciones que siguen nuestras variables predictivas en un histograma.

In [ ]:
utils.plot_histogram(data_train)



Generado el histograma, veamos que sucede con cada variable predictora:

*     Pregnancies: sigue una distribución normal.
*     Glucosa: sigue una distribución normal. Aparecen registros con glucosa igual a 0, cosa que no puede suceder en una persona,         por tanto, se consideran dato ruidoso.
*     Presión sanguinea: sigue una distribución normal. Sucede lo mismo que en el atributo glucosa, que aparecen datos ruidosos.
*     Espesor de la piel: vemos que,existe un número elevado de registros con valor 0, pero, tras el ruido, sigue una distribución          normal.
*     Insulina: ocurre exactamente lo mismo que con la variable "Espesor de la piel", pero esta presenta outliers.
*     Indice de Masa Corporal: sigue una distribución normal, pero existen registros con valor 0.
*     Diabetes heredada: sigue una distribución normal. Existen registros con valor 0, pero en este caso es algo coherente, porque existirán personas que no hereden la diabetes.
*     Edad:sigue una distribución normal.

Para confirmar que siguen una distribución normal, lo haremos con la gráfica Quantile-Quantile. Cuanto más cerca estén los puntos a la linea recta, mas parecida será su distrubición a la normal. (Para ver correctamente las representaciónes, pulsar en el boton ><, que aparece en la derecha tras ejectuar el código, para que se escondan los elementos, y volver a pulsarlo para ver correctamente las representaciones).


In [ ]:
import pylab
import scipy.stats as stats

a= list(X_train.columns.values)


for col in a:
    print(col)
    stats.probplot(X_train[col],dist="norm",plot=pylab)
    pylab.show()

Como podemos ver, las variables se asemejan con lo visto en los histogramas. Incluso con esta representación, vemos más claro los datos ruidosos y outliers en varias variables predictoras.

Hemos visto antes, que existen variables predictoras con datos ruidosos igual a 0, vamos a calcular el porcentaje que representan y decidiremos si imputar o eliminar dichas variables.

In [ ]:
a=X_train.loc[X_train['Glucose'] == 0].count()[1]
print((a/X_train.count()[1])*100)
a=X_train.loc[X_train['BloodPressure'] == 0].count()[1]
print((a/X_train.count()[1])*100)
a=X_train.loc[X_train['SkinThickness'] == 0].count()[1]
print((a/X_train.count()[1])*100)
a=X_train.loc[X_train['Insulin'] == 0].count()[1]
print((a/X_train.count()[1])*100)
a=X_train.loc[X_train['BMI'] == 0].count()[1]
print((a/X_train.count()[1])*100)

Visto esto, vemos que con la insulina existe un 46% de datos ruidosos, por lo que eliminaremos dicha variable, porque no aporta suficiente información. Lo mismo haremos con la variable "Espesor de la piel". Mientras que, con las otras 3 variables, haremos una imputación.

**Detección de Outliers**

En el apartado anterior, usando histogramas y diagramas Quantile-Quantile, vimos que algunas variables presentaban outliers. A continuación, realizaremos un estudio sobre ellos.

In [ ]:
a= list(X_train.columns.values)

for col in a:
    plt.title(col)
    plt.boxplot(X_train[col], vert=False)
    plt.show()

Para la detección de outliers, hemos utilizado los diagramas de bigotes. Un diagrama de bigote es una caja, formado por el primer cuartil (Q1), que representa el inicio de la caja y es el valor por debajo del cual se encuentran el 25% de los datos; el tercerl cuartil (Q3), representa el final de la caja y es el valor por debajo del cual se encuentran el 75% de los datos; mediana, es la línea que se encuentra dentro de la caja. Con ello, podemos observar que las distintas varibles predictoras contienen outliers.

*     Pregnancies: presenta tres outliers por encima del bigote superior.
*     Glucosa: presenta un outlier por debajo del bigote inferior.
*     Presión de la sangre: presenta varios outliers tanto por encima como por debajo.
*     Espesor de la piel: no presenta outliers.
*     Insulina: presenta una gran cantidad de outliers por encima del bigote superior.
*     Indice de Masa Coporal: presenta varios outliers tanto por encima como por debajo.
*     Diabetes heredada: presenta una gran cantidad de outliers por encima del bigote superior.
*     Edad: presenta outliers por encima del bigote superior.

En el apartado de preprocesamiento eliminaremos los outliers, con el objetivo de tener más limpios nuestros datos.


**Correlación de los datos**

En este apartado, vamos a estudiar la correlación de nuestras variables predictoras. Nuestro objetivo será seleccionar las que tengan correlación, para luego representarlos en un pairplot.

In [ ]:
correlation_mat=X_train.corr()
sns.heatmap(correlation_mat,annot=True)
plt.show()

Hemos generado una matriz cuadrada, que muestra la correlación entre cada par de variables predictoras. Además, muestra la medida de fuerza de asociación, de las cuales nos quedaremos con las correlaciones mayores y menores a 0.5, porque significa que existe correlación entre las variables.

In [ ]:
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
corr_fuerte=sorted_pairs[((sorted_pairs > 0.5) & (sorted_pairs < 1)) | (sorted_pairs < -0.5)]
print(corr_fuerte)

Vemos que sólo existe correlación entre las variables `Pregnancies` y `Edad`.

**Pairplot**

Realizaremos un pairplot, para observar que se distribuyen los registros, en base a la clase, para decidir que tipo de discretización realizaremos más adelante.

In [ ]:
utils.plot_pairplot(data_train, target="Outcome")

In [ ]:
data_train.describe(include="number")

Podemos ver que para realizar la clasificación no es una tarea sencilla, porque hay una gran cantidad de información. Al haber tantos datos, lo mejor es descretizar en varios intervalos que en 2, por tanto, discretizaremos en 3 intervalos.

 ># 1.3.  Preprocesamiento de datos

Una vez hecho el análisis exploratorio de datos, procederemos a realizar el preprocesamiento de datos. Con lo anteriormente visto, debemos imputar variables, eliminar outliers y realizar la discretización. Para ello nos ayudaremos de la creación de varios transformadores de datos, los cuales nos permitirán realizar las tareas descritas anteriormente.

#### Imputar Variables

Utilizaremos el transformador `SimpleImputer`. Este nos permitirá imputar las variables BMI,Glucose y BloodPressure, es decir, en estas 3 variables sustituiremos los 0 por la media correspondiente a cada atributo. Para ello debemos utilizar un `ColumnTransformer`, por el cual nos permite realizar la imputación a las variables indicadas.

In [ ]:
variables=['BMI','Glucose','BloodPressure']

imputador= make_column_transformer(
    (make_pipeline(
        SimpleImputer(missing_values=0, strategy='median')
    ),variables))

#### Eliminar columnas

Visto en el análisis exploratorio, que las variables Insulin y SkinThickness tenían más del 20% de datos perdidos, eliminaremos dichas variables, porque no aportan información. Para ello debemos implementar nuestra propio transformador, el cual será dropVar, el cual se basa en eliminar las columnas que le lleguen como argumento. Hemos utilizado el siguiente [tutorial](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156) para implementar la clase correctamente.

In [ ]:
class dropVar():
    
    def __init__(self, column):
        self.column=column
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, x, y=None):
        cop=x.copy()
        cop.drop(self.column,axis=1)
        return cop

#### Eliminar Outliers

Vimos en el análisis exploratorio, concretamente en los diagramas de cajas y bigotes, que existen outliers en nuestras variables predictoras. Para reducir los outliers, implementaremos un método que se encargue de eliminar dichos outliers. Utilizaremos el transformador `FunctionSampler`, el cual utilizará el método outlier_rejection, para llevar a cabo la reducción de outliers.

In [ ]:
def outlier_rejection(X, y):
    
    model = IsolationForest(max_samples=100,
                            contamination=0.4,
                            random_state=seed)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

#### Discretizador

En la explicación de la práctica se nos explicó la importancia de discretizar variables, ya que, nos permitía que modelos lineales resuelvan problemas no lineales. Para ello utilizaremos el transformador `KBinsDiscretizer`. En el análisis de datos, no se apreciaba la forma más efectiva para dividir los datos, porque había una gran cantidad de registros. Al tener tanta información dispersada, lo mejor será construir 3 intervalos de igual anchura.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, strategy="uniform")

># 1.4 Algoritmos de clasificación

Llegados a este punto, debemos de declarar los algoritmos de clasificación que se nos pedía para esta práctica. Estos algoritmos son Zero-R y Árbol de Decisión (con y sin descritización).

## Zero-R

El algoritmo de Zero-R predice sobre la clase o categoría principal, del conjunto entrenamiento, a los nuevos casos.

In [ ]:
zero_r=DummyClassifier(strategy="most_frequent")

## Algoritmo CART (Classification and Regression Trees): Inducción de árboles de decisión

Es un algoritmo que sirve para clasificar utilizando particiones sucesivas. Es apropiado cuando hay un número elevado de datos, aportando un carácter descriptivo que permite entender e interpretar fácilmente las decisiones tomadas por el modelo. Primero creamos un árbol sin discretizar.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

### Árbol discretizado

Para realizar un clasificador discretizado, utilizaremos los Pipelines. Este toma como parámetros la lista de transformadores a aplicar al conjunto de datos y, al final de este, el estimador a utilizar. Para ello nuestros transformadores serán los descritos en el apartado 4. Crearemos tres pipelines, uno aplicando todos los transformadores, otro donde se apliquen todos menos el discretizador. Lo anterior nos servirá para evaluar nuestros modelos.

In [ ]:
lista= ['Insulin','SkinThickness']

In [ ]:
discretize_tree_model = make_pipeline(dropVar(lista),imputador,FunctionSampler(func=outlier_rejection),discretizer, tree_model)

In [ ]:
tree_model_cleaned= make_pipeline(dropVar(lista),imputador,FunctionSampler(func=outlier_rejection), tree_model)

># 1.5. Evaluación de modelos.

Por último, nos queda entrenar nuestros modelos y validarlos. Comenzaremos con el Zero-R

In [ ]:
utils.evaluate(zero_r,
               X_train, X_test,
               y_train, y_test)

Realizaremos lo mismo con los árboles de clasificación.

In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.evaluate(tree_model_cleaned,
               X_train, X_test,
               y_train, y_test)

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

De estas validaciones podemos sacar las siguientes conclusiones:
* El algoritmo de árboles de decisión funciona mejor que el Zero-R, como era de esperar.
* El modelo de árbol de decisión, con los transformadores de limpieza, obtiene un 69% lo cual supone un 1% menos que el modelo de árbol de decisión. Esto significa que empeoramos el modelo realizando el proceso de limpieza, puede ser que se elimine información valiosa. Además, influye el factor de la semilla a la hora de realizar las divisiones en el árbol de decisión.
* El modelo de árbol de decisión, discretizado y con todos los transformadores, obtiene un 71%, por tanto, obtenemos un mejor modelo que los 3 anteriores descritos. Esto se debe a que discretizamos de forma eficiente nuestro conjunto de datos. Al igual que antes, el factor de la semilla influye.

# 2. WINCONSIN-DATA

# 2.1. Acceso y almacenamiento de datos

In [ ]:
seed = 17102

El conjunto de datos a tratar ahora es `wisconsin`. En este conjunto se detalla una bbdd en la que se examinan características de las células de un cáncer benigno o maligno. Encontramos 569 casos donde la variable objetivo, que denomina el tipo de cáncer`diagnosis`, puede ser:

* `M = malignant`: Maligno
* `B = benign`: Benigno

Para determinar el tipo de cáncer se han tomado varias de las características que hacen referencia a la media, el error estándar y "peor" o mayor (media de los tres valores más grandes) para cada imagen de las células, resultando en 30 variables predictoras, todas ellas continuas:

* `radius`: radio, distancia media desde el centro hasta los puntos del perímetro.
* `texture`: textura, desviación estándar de los valores de la escala de grises.
* `perimeter`: perímetro, suma de las longitudes del contorno de las figuras/formas.
* `area`: concepto métrico que puede permitir asignar una medida a la extensión de una superficie.
* `smoothness`: suavidad, variación local en longitudes de radio.
* `compactness`: compacidad, (perímetro^2 / area - 1.0)
* `concavity`: concavidad, severidad de las porciones cóncavas del contorno
* `concave points`: puntos cóncavos, número de puntos cóncavos del contorno
* `symmetry`: simetría
* `fractal dimension`: dimensión fractal, ("aproximación de la costa" - 1)

El objetivo sería clasificar nuevos casos como malignos o benignos en función de sus propiedades.

Cargamos el conjunto de datos `wisconsin`

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = 'id'
target = 'diagnosis'

data = utils.load_data(filepath, index, target)

Ahora comprobamos que la base de datos se ha cargado correctamente cargando los 5 ejemplos aleatorios

In [ ]:
data.sample(5, random_state=seed)

Podemos observar que por error se nos genera una última columna que, comprobando la descripción de nuestra base `wisconsin`,se debe a que en la última variable introduce una coma al final, lo que acaba generando una especie de "variable vacía" que eliminaremos

In [ ]:
data = data.drop(columns=['Unnamed: 32'])
data.sample(5, random_state=seed)

Ahora separaemos nuestro conjunto de datos en 2, uno con las variables predictoras (X) y otro con la variable objetivo `diagnosis` (y).

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

De nuevo, comprobamos que todo esté correcto

In [ ]:
X.sample(5, random_state=seed)

In [ ]:
y.sample(5, random_state=seed)

Para el análisis exploratorio hemos dividido nuestro conjunto de datos en 2 con los siguientes porcentajes:

* Una muestra de entrenamiento (típicamente, 70%)
* Una muestra de prueba (típicamente, 30%)


Ahora aleatorizamos nuestros datos e iniciamos el proceso de holdout

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

De nuevo, comprobamos que esta división se ha llevado a cabo correctamente

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

# 2.2. Análisis exploratorio de datos

Antes de cualquier operación es indispensable identificar:
* Número de casos
* Número de variables
    * Tipo de las variables: Continuas (t.c.c. numéricas) o discretas (t.c.c. categóricas)
    

In [ ]:
print("(número de casos, númerero de variables): ",data.shape,'\n') 
print("información de las varibles:\n")
print(data.info(memory_usage=False))
print("\nvalores de la variable objetivo: \n",y.cat.categories)


Como ya hemos comentado, tenemos **569 casos** y **31 variables**, 30 discretas (`float64`) y 1 categórica (`category`) que es nuestra variable objetivo cuyos valores pueden ser `B`o `M`

A parte también observamos que no existen nulos ya que la cuenta de todas las variables es el total, 569

### Visualización de las variables

Para empezar con la visualización, vamos a observar la diferencia de casos que hay en la partición de entrenamientos referidas a la variable `diagnosis`

In [ ]:
#Creamos una variable auxiliar con todos los datos para las gráficas, como están ordenadas de la misma manera simplifica el resultado
X_y_train = X_train[0:]
X_y_train['diagnosis'] = y_train
utils.plot_barplot(X_y_train)

In [ ]:
y_train.describe()

Podemos observar que abundan más casos donde el cáncer acaba siendo benigno (63% = 250) que maligno (37% = 148)

Como ya hemos mencionado, esta bbdd tiene 30 variables, que realmente son 10, pero resultan en 30 debido a que se calculan en relación a la media, el error y la media de los 3 valores mas grandes. Luego para verlo mejor, dividiremos esta sección en 10 graficas, para que de esta forma puedan ser más visuales.

In [ ]:
cols = list(X_train.columns)

X_train1 = X_train[[cols[0]] + [cols[10]]+ [cols[20]]]
utils.plot_histogram(X_train1)

In [ ]:
X_train2 = X_train[[cols[1]] + [cols[11]] + [cols[21]]]
utils.plot_histogram(X_train2)

In [ ]:
X_train3 = X_train[[cols[2]] + [cols[12]] + [cols[22]]]
utils.plot_histogram(X_train3)

In [ ]:
X_train4 = X_train[[cols[3]] + [cols[13]]+ [cols[23]]]
utils.plot_histogram(X_train4)

In [ ]:
X_train5 = X_train[[cols[4]] + [cols[14]]+ [cols[24]]]
utils.plot_histogram(X_train5)

In [ ]:
X_train6 = X_train[[cols[5]] + [cols[15]]+ [cols[25]]]
utils.plot_histogram(X_train6)

In [ ]:
X_train7 = X_train[[cols[6]] + [cols[16]]+ [cols[26]]]
utils.plot_histogram(X_train7)

In [ ]:
X_train8 = X_train[[cols[7]] + [cols[17]]+ [cols[27]]]
utils.plot_histogram(X_train8)

In [ ]:
X_train9 = X_train[[cols[8]] + [cols[18]]+ [cols[28]]]
utils.plot_histogram(X_train9)

In [ ]:
X_train10 = X_train[[cols[9]] + [cols[19]]+ [cols[29]]]
utils.plot_histogram(X_train10)

Las conclusiones que podemos sacar de estas gráficas son varias, comenzando con que todas las variables siguen más o menos (en algunos casos necesitamos ampliar para verlo) que las distribuciones tienen una tendencia en forma de campana, a parte, también podemos ver que las gráficas `mean` son semejantes entre si, lo mismo ocurre con `se` y `worst`.

También podemos observar que los mayores outliers se encuentran en las gráficas de `mean` y `worst`

Antes de echar un vistazo a las gráficas de discretización, vamos a inspeccionar todas variables para ver cómo evoluciona el cáncer dependiendo de los valores de dichas variables.

In [ ]:
X_train.describe(include="number")

In [ ]:
def _plot_barplot2(variable, color):
    fig = px.histogram(X_y_train, x=variable, color=color)
    fig.show()

categorical_data = utils._filter_numerical_data(X_y_train)
var = categorical_data.columns
data = widgets.fixed(categorical_data)

widgets.interact(_plot_barplot2, variable=var,color="diagnosis")

Lo que primeramente podemos observar de estas gráficas es que los casos de beningno alcanzan valores `count` más altos en general que los de maligno, esto se debe a lo ya analizado anteriormente y es que tenemos más casos benignos que malignos luego es algo norlmal.

Respecto a los valores hay algo que siguen la totalidad de las variables, esto es que a valores bajos siempre predominan los casos de benigno, mientras que a valores altos tenemos 2 opciones bastante reñidas, una es que más o menos estén en igualdad de casos o que el caso maligno predomine (hay algunas excepciones donde en valores altos predomina los casos benignos estos son `fractal_dimension_mean`, `fractal_dimension_se`), esto podría influir entonces a la hora de discretizar.

Otra cosa bastante observable es que las gráficas describen figuras en forma de campana y en muchos casos también podemos darnos cuenta de como ambos casos, beningno y maligno, coinciden a la hora de crecer, el punto más alto y la disminución, como ocurre en `smoothness_mean`

In [ ]:
X_y_train

En las próximas gráficas podemos ver que las diagonales se corresponden con las anteriores, luego ya, llevamos algo de adelanto para la hora de discretizar. Nos quedaría el resto, luego:

**¿Cuáles serían buenas variables para discretizar?**

In [ ]:
#MEAN
cols = list(X_y_train.columns)
X_y_train11 = X_y_train[[cols[-1]] + cols[0:10]]
utils.plot_pairplot(X_y_train11, target="diagnosis")

En el caso de `mean` podríamos discretizar por`radius_mean`, `perimeter_mean`, `area_mean` y `concave pointes_mean` (eje x) ya que al combinarlas con el resto de variables, estas 4 provocan que a mayor número de ellas, el cáncer sea maligno y a menor benigno, luego podriamos discretizar bastante bien con ellas. Además de que prácticamente no hay muchos outliers en estos casos, cosa que por ejemplo si ocurre en bastantes combinaciones de la variable `concavity_mean` (eje x).

In [ ]:
#SE
X_y_train12 = X_y_train[[cols[-1]] + cols[10:20]]
utils.plot_pairplot(X_y_train12, target="diagnosis")

Respecto a los casos `se` podemos observar que de principio existen bastantes outliers en la mayoría de las gráficas, y que básicamente las discretizaciones podrían salir de`perimeter_se` y `area_se` (eje x) junto con todas sus combinaciones. Además algunas discretizaciones adicionales también podrían ser algunas combinaciones de `radius_se` (aunque nos puedan empeorar algo los outliers).

In [ ]:
#WORST
X_y_train13 = X_y_train[cols[20:]]
utils.plot_pairplot(X_y_train13, target="diagnosis")

Por último los `worst`, existen batantes casos por los que se podría realizar una buena discretización, por ejemplo todas las combinaciones de las variables `radius_worst`, `perimeter_worst`, `area_worst`, `compactness_worst`, `concave points_worst`.

A parte, ya no las voy a nombrar porque son muchísimas, el resto también tiene combinaciones de variables muy buenas para discretizar como es el caso de por ejemplo la combinación (x,y) (`texture_worst`,`area_worst`).

# 2.3. Preprocesamiento de datos

Dentro del preprocesamiento de datos, podemos destacar las siguientes tareas:

* Limpieza de datos (imputación de valores perdidos, suavizado del ruido, etc.)
* Integración de datos (a partir de múltiples fuentes)
* Transformación de datos (normalización, construcción, etc.)
* Reducción de datos (discretización de variables numéricas, selección de variables, selección de instancias, etc.)

Sin embargo siguiendo las directrices de la práctica, solo nos centraremos en la discretización de variables numéricas

### Discretización

Observando las gráficas, una discretización por anchura puede que no sea una buena manera, luego probaremos con frecuencia y a parte divideremos en más intervalos ya que viendo las gráficas al divirlas en más podremos dejar casos más aislados ya que la mezcla de variables, menigno y maligno, se encuentra más o menos cerca del centro y en los principios y finales de las gráficas suele predominar una variabble

In [ ]:
discretizer = KBinsDiscretizer(n_bins=4, strategy="quantile")

# 2.4. Algoritmos de clasificación

## Zero-R

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

## Árbol de decisión

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

### *Pipeline*

In [ ]:
discretize_tree_model = make_pipeline(FunctionSampler(func=outlier_rejection),discretizer, tree_model)

In [ ]:
discretize_tree_model_1 = make_pipeline(discretizer, tree_model)

# 2.5. Evaluación de modelos

## Zero-R

In [ ]:
utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

## Árbol de decisión sin discretización


In [ ]:
utils.evaluate(tree_model,
               X_train, X_test,
               y_train, y_test)

## Árbol de decisión con discretización y filtrado

In [ ]:
utils.evaluate(discretize_tree_model,
               X_train, X_test,
               y_train, y_test)

## Árbol de decisión con discretización sin filtrado

In [ ]:
utils.evaluate(discretize_tree_model_1,
               X_train, X_test,
               y_train, y_test)

En estas evaluaciónes, podemos sacar las siguientes conclusiones:
* El modelo obtenido del Zero-R es el peor, tal y como esperabamos, por su funcionamiento.
* El árbol sin discretización obtiene un 95% de accuracy, siendo mejor que los modelos discretizados y con filtrado.
* El árbol discretizado sin filtrado obtiene mejor modelo que el filtrado. Puede darse que, eliminando outliers afectamos a la discretización, por tanto, sin la eliminación de estos se discretiza mejor el conjunto de datos.

Utilizar `Accuracy` no es lo más apropiado. Estamos diagnosticando una enfermedad, encima muy grave, no sería lo mismo fallar diagnosticando un cáncer benigno que maligno. Luego un método de validación sensible al coste, donde se penalizara más el equivocarnos al diagnosticar un cáncer, que debe ser maligno, tendría más sentido que utilizar el `Accuracy`.